## First theoretical calculations for an Achromatic Beam Line in First & Second Order 

## C. Mussolini and N. Charitonidis (for internal use only)

## TRANSPORT manual to be taken into consideration : https://cds.cern.ch/record/133647/files/CERN-80-04.pdf
##  And mathematical framework for the matrices theory : http://inspirehep.net/record/187522/files/slac-r-075.pdf

In [1]:
#This program calculates the drift lengths given the position of a first quadrupole and the 
#focusing strengths of the following ones.

#It does this by adding dictionaries for each quadupole element to a list, multiplying
#through the list the R matrices and solving the 11, 12 elements to get hte needed drifts.

#Then works out the R matrix at every step of the multiplication of matrices, adds to a list
#and plots this at the end.

In [2]:
%matplotlib inline 
#notebook

from math import *
import holoviews as hv
import numpy as np
import matplotlib.pyplot as plt
import sympy as sy
from scipy.linalg import block_diag
sy.init_printing()
#hv.notebook_extension('bokeh', 'matplotlib')

ModuleNotFoundError: No module named 'holoviews'

In [ ]:
# matrices are defined according to what they are in x direction. 
def Create_Drift_Space(drift_lenght_symbol): #create symbolic dictionary for drift space 
    
    Ld = drift_lenght_symbol
    magnet = {'matrix_x' : sy.Matrix([[1,Ld],[0,1]]),
              'matrix_y' : sy.Matrix([[1,Ld],[0,1]]),
              'type_x'   : 'drift',
              'type_y'   : 'drift',
              'length' : Ld
             }
    return(magnet)

def Create_Focusing_Magnet(focusing_symbol, aperture): #create symbolic dictionary for thin foc quad
    
    f = focusing_symbol      
    magnet = { 'matrix_x' : sy.Matrix([[1,0],[-1/f, 1]]), 
               'matrix_y' : sy.Matrix([[1,0],[1/f, 1]]), 
               'apt' : aperture,
               'type_x': 'focus',
               'type_y': 'defocus',
               'length' : 0
             }
    return(magnet)
    
def Create_Defocusing_Magnet(focusing_symbol, aperture): #create symbolic dictionary for thin defoc quad
    
    f = focusing_symbol 
    magnet = { 'matrix_x' : sy.Matrix([[1,0],[1/f, 1]]),
               'matrix_y' : sy.Matrix([[1,0],[-1/f, 1]]), 
               'apt' : aperture,
               'type_x': 'defocus',
               'type_y': 'focus',
               'length' : 0
             }

    return(magnet)

In [ ]:
def Full_R_Matrix(R_matrix_x, R_matrix_y): #This combines the x and y R matrices to easily extract the components
    
    R_matrix = sy.Matrix(block_diag(R_matrix_x, R_matrix_y))
    
    return(R_matrix)

def Matrix_eval(matrix, var_1, var_2, var_3, var_4, distance, focusing, a, b): #Evaluates a symbolic matrix into a numerical one
    
    expr = sy.lambdify([var_1, var_2, var_3, var_4], matrix, "numpy") 
    values = expr(distance, focusing, a, b)
    
    return(values)
    
def Desired_Parms(R_matrix): #This extracts the ratios of R values we are interested in
    
    comp1 =  R_matrix[0,0]
    comp2 =  R_matrix[0,1]
    comp3 =  R_matrix[2,2]
    comp4 =  R_matrix[2,3]
    
    return(comp1, comp2, comp3, comp4)
    
def Construct_line(elements): #This function, given a list of lenses and drift spaces, multiplies them out 
                              #and returns the R matrix for each step.
    
    partial_Rs = []
    R_matrix = sy.Matrix([[1,0],[0,1]])
    
    for component in elements:
        R_matrix = component@R_matrix
        partial_Rs.append(R_matrix)
        
    return(partial_Rs)  
  
def Locate_Arrows(list_of_dict, axis):#given the dictionaries of quadrupoles, pulls out the type and
                                      #position, then returns the position of arrowheads to plot.
    position = 0
    y = []    
    dy = []
    
    for dictionary in list_of_dict:
        #choose type of magnet, if docusing arrows inwards, opposite if defocusing.
        if dictionary['type_' + axis] == 'defocus' : 
            #start of arrow at aperture/2 (for symmetry), and length for sake of focusing to work out
            y.append(dictionary['apt']/2)
            dy.append(dictionary['apt']/2)
            
        elif dictionary['type_' + axis] == 'focus':
            #start of arrow at aperture/2 (for symmetry), - to go inwards
            y.append(dictionary['apt'])
            dy.append(-dictionary['apt']/2)
            
    return(y,dy)

In [ ]:
#enter the paraeters wanted here. Can be easily expanded
foc = 1
init_dist = 1
aperture =  0.1#5#500 * 10 ** -3

In [ ]:
#define a few symbolic variables, f = focus, Ld = drift lenght, a = distance before first quad
#b = distance from last quad to observing point.

f = sy.Symbol( 'f', real=True)
Ld = sy.Symbol( 'Ld', real=True)
a = sy.Symbol( 'a', real=True)
b = sy.Symbol( 'b', real=True)

#create the dictionaries for each component that will be used or we are interested in

focus = Create_Focusing_Magnet(f, aperture)
defocus = Create_Defocusing_Magnet(f, aperture)
double_focus = Create_Focusing_Magnet(2*f, aperture)
double_defocus = Create_Defocusing_Magnet(2*f, aperture)
drift = Create_Drift_Space(Ld)
first_drift = Create_Drift_Space(a)
last_drift = Create_Drift_Space(b)

#create a list of variables we have defined and ones we have used but need to calculate, 
#important to keep the symbolic ones and the real ones at the same position in each different
#list. ---Maybe a dictionary could have been better, with keys as symbolic variables and entry
#the real values...

values_of_vars = [foc, init_dist]
list_of_variables_decided = [f, a]
list_of_variables_to_find = [Ld, b]

list_of_variables_used = list_of_variables_decided + list_of_variables_to_find

In [ ]:
#define your beamline by taking the componets and putting them in a list.

#dicts_used = [ first_drift, focus,  drift, defocus, drift, focus, drift, defocus, drift, focus, drift, defocus, last_drift]
#dicts_used  = [ first_drift, focus,  drift, defocus, drift, focus, drift, defocus, last_drift]
dicts_used  = [ first_drift, defocus,  drift, focus, drift, defocus, drift, focus, last_drift]

#Here we pull from the dictionaries the x and y parts individually
elements_used_x = []
elements_used_y = []

for dictionary in dicts_used:

    elements_used_x.append(dictionary['matrix_x'])
    elements_used_y.append(dictionary['matrix_y'])
    
#This multiplies together the matrices given above, extractiong the R matrix for each step

R_in_x = sy.simplify(Construct_line(elements_used_x))
R_in_y = sy.simplify(Construct_line(elements_used_y))

In [ ]:
#Here you look at element 2,1 at the end of the beamline and solve for drift lenght in terms of
#the focusing strength. Various if statements to pick the ideal lenght value. 
#Must be always positive, If solution independant of Ld, throuw warning, if en solutions = 1 
#and its positive, great, if more olutions, then pick the smallest positive one.

focus = sy.lambdify([f, a], R_in_x[-1][1,0], 'numpy')
solutions_f = sy.solve(focus(foc, init_dist), Ld)
if len(solutions_f)==0:
    L_D = 100000000
    print("Warning: solutions are independant of the focus!")
elif len(solutions_f)==1 and solutions_f[0]>=0 :
    L_D = sy.N(solutions_f[0]) #N is sympy for evaluating
else:
    L_D = sy.N(min(n for n in solutions_f if n > 0))

In [ ]:
#here you look at element 1,2, and do the sme as above, but solve for focusing using the peviously 
#found drift length. If statments play the same role

elem12 = sy.lambdify([Ld, f, a], R_in_x[-1][0,1], 'numpy')
solutions = sy.solve(elem12(L_D, foc, init_dist))
if len(solutions)==0:
    fin_dist = 1000000
    print("Warning: solutions are independant of the final distance!")
elif len(solutions)==1 and solutions[0]>=0:
    fin_dist = sy.N(solutions[0])
else:
    fin_dist = sy.N(min(n for n in solutions if n > 0)) 

In [ ]:
#add the just found values to the list of varialbes we have values for now

values_of_vars.append(L_D)
values_of_vars.append(fin_dist)

In [ ]:
#evaluare with the values just found above/inputted

R_in_x_eval = Matrix_eval(R_in_x, *list_of_variables_used, *values_of_vars)
R_in_y_eval = Matrix_eval(R_in_y, *list_of_variables_used, *values_of_vars)

In [ ]:
#Works out the positions of all the magnets, given the drift spaces jsut calculated.
#Create a list for the positions of magntes AND drift, and one only for the magnets.
#If you move forward after adding a component, then had a drift, otherwise have a quadrupole
#add to lists accordingly. 0 in pos_magnets is just for if statment later, for pos_elements
#its to start beam at origin.

pos_magnets = [0]
pos_elements = [0]
i = 0

for dictionary in dicts_used:
    
    #Work out teh lenth of element, then new position, then add for both drift and magnets
    
    val = sy.lambdify(list_of_variables_used, dictionary['length'])
    new = val(*values_of_vars) + pos_magnets[-1]
    pos_elements.append(new)
    
    #if position is not same as last position, drift finished, so will have a magnet there.
    #other clauses are for if there is no drift before magnet or if it is at the very end.
    
    if (new != pos_magnets[-1] and i != len(dicts_used)-1) or (i == 0 and new == 0):  
        pos_magnets.append( new )
    i += 1
    
#remove the placeholder 0    

pos_magnets = pos_magnets[1:]

In [ ]:
#Here we pull out the ratios we are interested in. 0s are in o start at beginning of beamline
ratios = []
R11 = [1]
R12 = [0]
R33 = [1]
R34 = [0]

for x, y in zip(R_in_x_eval, R_in_y_eval):
    ratios.append(Desired_Parms(Full_R_Matrix(x,y)))
    
for item in ratios:
    R11.append(item[0])
    R12.append(item[1])
    R33.append(item[2])
    R34.append(item[3])

In [ ]:
print("R matrix values are:")
if fin_dist > 0:
    print("[ %3.2f, %3.2f, %3.2f, %3.2f ]" %(R11[-1], R12[-1], R33[-1], R34[-1]))

else:
    print("Not allowing a telescope, but values would be")
    print("[ %3.2f, %3.2f, %3.2f, %3.2f ]" %(R11[-1], R12[-1], R33[-1], R34[-1]))
    
print("Focusing strength: %5.2f" %(foc))
print("Initial length: %8.2f" %(init_dist))
print("Drift length: %10.2f" %L_D)
print("Final lenght: %10.2f" %(fin_dist) )
print("Total lenght: %10.2f" %pos_elements[-1])

In [ ]:
#Plot the various R values. Each element is at an integer value, but this can be easily changed later

plt.plot(pos_elements, R11, 'g', label = "R11" )
plt.plot(pos_elements, R12, 'b', label = "R12")
xy, xdy = Locate_Arrows(dicts_used, 'x')

#add the arrows, with x as position of magnets, y and dy given by lovate arrows, and dx = 0 for all

for x,y,dx,dy in zip(pos_magnets, xy, np.zeros(len(pos_magnets)), xdy):
    plt.arrow(x, y, dx, dy, color = 'r', head_width=0.1, head_length=0.2)
    plt.arrow(x, -y, dx, -dy, color = 'r', head_width=0.1, head_length=0.2)

plt.plot([0, max(pos_elements)],[0, 0], '--r')
plt.xlim(left=0)
plt.ylim(-10,10)
plt.legend(loc = 2)
plt.xlabel('Distance [m]')
plt.show()

plt.plot(pos_elements, R33, 'g', label = "R33")
plt.plot(pos_elements, R34, 'b', label = "R34")
yy, ydy = Locate_Arrows(dicts_used, "y")

for x,y,dx,dy in zip(pos_magnets, yy, np.zeros(len(pos_magnets)), ydy):
    plt.arrow(x, y, dx, dy, color = 'r', head_width=0.1, head_length=0.2)
    plt.arrow(x, -y, dx, -dy, color = 'r', head_width=0.1, head_length=0.2)

plt.plot([0, max(pos_elements)],[0, 0], '--r')
plt.ylim(-10,10)
plt.xlim(left=0)
plt.legend(loc = 2)
plt.xlabel('Distance [m]')
plt.show()